基于大模型启发式数据库图搜索算法
一、构建数据库图
节点（Nodes）：图中的每个节点代表数据库中的一个元素，可以是“库名”、“表名”或者“字段”。

边（Edges）：边连接节点之间，表示它们之间的关系。边的权重可以由注释内容的相似度、关联度等因素确定，帮助启发式搜索判断不同节点之间的相关性。

数据库图结构
库名节点（Database Node）：
包含数据库的基本信息（如库名、描述等）。

表名节点（Table Node）：
包含表的名称、描述和相关的字段。

字段节点（Field Node）：
包含字段名、字段描述、字段的注释。

连接条件
注释中显式的连接条件
通过相同字段匹配的连接条件
让大模型根据两张表的连接字段提取连接信息

库名与表名之间的连接：表示某个库下包含了哪些表。
表名与字段之间的连接：表示某个表包含了哪些字段。
字段之间的连接：根据字段的注释内容进行连接。如果两个字段的注释有很高的相似度或者存在业务逻辑上的联系（如公司代码与公司名称），则在图中连接这些字段。

库名-表名层： 连接每个库名与它所包含的表。

表名-字段层： 连接表与它包含的字段，每个表下的字段形成一个子图。

字段-字段层： 根据字段的注释内容（column_description），计算字段之间的相似度或相关性，建立边连接。

二、图搜索算法



三、启发式图搜索算法
启发函数


In [2]:
import pandas as pd
import jieba
import re
import requests
import json
from zhipuai import ZhipuAI
from collections import Counter
from tqdm import tqdm
import time
from util import utils

In [3]:
Access_Token = 'a30d01d4630a43b087a1f9851680902a'  # Competition team Token, used to access the competition database
MODEL = "glm-4-air"  # Default large model used; this solution uses the GLM-4-PLUS model entirely
client = ZhipuAI(api_key='f689043de7886e8c604802325fa16392.9m8y9PP2XBKP9ZJy')

# Preprocess the competition questions here
question_data_path = r'./data/raw/question.json'
data_dict_path = "./data/raw/data_dict.xlsx"
df1 = pd.read_excel(data_dict_path, sheet_name='库表关系')
df2 = pd.read_excel(data_dict_path, sheet_name='表字段信息')
file_path = './data/raw//all_tables_schema.txt'

In [4]:
# df1['库表名英文'] = df1['库名英文'] + '.' + df1['表英文']
# df1['库表名中文'] = df1['库名中文'] + '.' + df1['表中文']

# database_name = list(df1['库名中文'])
# table_name = list(df1['表中文'])
# table_name_en = list(df1['表英文'])
# database_table_ch = list(df1['库表名中文'])
# database_table_en = list(df1['库表名英文'])
# database_table_en_zs = {'库表名': database_table_en, '对应中文注释说明': table_name}
# database_table_map = df1.set_index('库表名中文')['库表名英文'].to_dict()

# database_L = []
# database_L_zh = []
# for i in table_name_en:
#     df3 = df2[df2['table_name'] == i]
#     name = df1[df1['表英文'] == i]['库表名英文'].iloc[0]
#     column_name = list(df3['column_name'])
#     column_name_zh = list(df3['column_description'])
#     column_name_2 = list(df3['注释'].dropna())

#     dict_1 = {'数据表名': name, '列名': column_name, '注释': column_name_2}
#     dict_2 = {'数据表名': name, '列名': column_name, '列名中文描述': column_name_zh, '注释': column_name_2}
#     database_L.append(dict_1)
#     database_L_zh.append(dict_2)

In [11]:
sql = """
SELECT DISTINCT a.InnerCode
FROM AStockMarketQuotesDB.CS_StockPatterns AS a
JOIN AStockMarketQuotesDB.CS_StockPatterns AS b
  ON a.InnerCode = b.InnerCode
  AND DATE(a.TradingDay) = DATE_SUB(DATE(b.TradingDay), INTERVAL 1 DAY)
JOIN AStockMarketQuotesDB.QT_DailyQuote AS c
  ON a.InnerCode = c.InnerCode
  AND DATE(a.TradingDay) = DATE(c.TradingDay)
JOIN AStockMarketQuotesDB.QT_DailyQuote AS d
  ON b.InnerCode = d.InnerCode
  AND DATE(b.TradingDay) = DATE(d.TradingDay)
WHERE a.IfHighestHPriceRY = 1
  AND b.IfHighestHPriceRY = 1
  AND c.HighPrice > 75
  AND d.HighPrice > 75
  AND DATE(a.TradingDay) = '2021-12-24'
  AND DATE(b.TradingDay) = '2021-12-23';
"""

utils.exec_sql(sql=sql)

'{\n  "success": true,\n  "data": [],\n  "count": 0\n}'